In [ ]:
from tvb.simulator.simulator import Simulator
from tvb.datatypes.connectivity import Connectivity
from tvb.contrib.inversion.sbiInference import sbiModel, sbiPrior

import tvb.simulator.models
import tvb.simulator.integrators
import tvb.simulator.coupling
import tvb.simulator.monitors

import matplotlib.pyplot as plt
import numpy as np
import torch
import math
import arviz as az
import pickle

%load_ext autoreload
%autoreload 2

In [ ]:
#run_ids = [
#    "2022-10-15_11-37-56-982207_instance",
#    "2022-10-15_11-37-57-045966_instance",
#    "2022-10-15_11-37-57-068800_instance",
#    "2022-10-15_11-37-57-077944_instance"
#]

#run_ids = [
#    "2022-10-15_18-38-18-522749_instance",
#    "2022-10-15_18-38-18-543724_instance",
#    "2022-10-15_18-38-18-564449_instance",
#    "2022-10-15_18-38-18-570811_instance",
#]

#run_ids = [
#    "2022-10-16_11-58-03-990396_instance",
#    "2022-10-16_11-58-04-005395_instance",
#    "2022-10-16_11-58-04-012163_instance",
#    "2022-10-16_11-58-04-014874_instance"
#]

#run_ids = [
#    "2022-10-16_22-46-26-433463_instance",
#    "2022-10-16_22-46-26-442912_instance",
#    "2022-10-16_22-46-26-467506_instance",
#    "2022-10-16_22-46-26-535119_instance"
#]

#run_ids = [
#    "2022-10-17_12-58-17-917167_instance",
#    "2022-10-17_12-58-17-937040_instance",
#    "2022-10-17_12-58-18-298925_instance",
#    "2022-10-17_12-58-18-303247_instance"
#]

#run_ids = [
#    "2022-10-17_18-30-34-971510_instance",
#    "2022-10-17_18-30-34-987136_instance",
#    "2022-10-17_18-30-35-214880_instance",
#    "2022-10-17_18-30-35-261021_instance"
#]

#run_ids = [
#    "2022-10-18_09-12-17-228856_instance",
#    "2022-10-18_09-12-17-251964_instance",
#    "2022-10-18_09-12-17-268330_instance",
#    "2022-10-18_09-12-17-293248_instance"
#]

#run_ids = [
#    "2022-10-18_09-19-32-373454_instance",
#    "2022-10-18_09-19-32-489723_instance",
#    "2022-10-18_09-19-32-540591_instance",
#    "2022-10-18_09-19-32-647061_instance"
#]

#run_ids = [
#    "2022-10-19_23-53-13-821369_instance",
#    "2022-10-19_23-53-13-845648_instance",
#    "2022-10-19_23-53-13-851198_instance",
#    "2022-10-19_23-53-13-859087_instance"
#]

#run_ids = [
#    "2022-10-19_23-53-14-055834_instance",
#    "2022-10-19_23-53-14-062800_instance",
#    "2022-10-19_23-53-14-073067_instance",
#    "2022-10-19_23-53-14-108728_instance"
#]

#run_ids = [
#    "2022-10-20_09-53-02-314927_instance",
#    "2022-10-20_09-53-02-359245_instance",
#    "2022-10-20_09-53-02-428250_instance",
#    "2022-10-20_09-53-02-545439_instance"
#]

#run_ids = [
#    "2022-10-20_11-44-37-689954_instance",
#    "2022-10-20_11-44-37-717804_instance",
#    "2022-10-20_11-44-37-722122_instance",
#    "2022-10-20_11-44-37-724352_instance"
#]

#run_ids = [
#    "2022-10-20_20-9-06-621622_instance",
#    "2022-10-20_20-9-06-627269_instance",
#    "2022-10-20_20-9-06-642803_instance"
#]

#run_ids = [
#    "2022-10-20_21-52-18-643971_instance",
#    "2022-10-20_21-52-18-668999_instance",
#    "2022-10-20_21-52-18-710365_instance",
#    "2022-10-20_21-52-18-721567_instance"
#]

#run_ids = [
#    "2022-10-26_22-47-15-635586_instance",
#    "2022-10-26_22-47-15-635789_instance",
#    "2022-10-26_22-47-15-656023_instance",
#    "2022-10-26_22-47-15-669513_instance"
#]

#run_ids = [
#    "2022-10-26_22-47-16-100078_instance",
#    "2022-10-26_22-47-16-216359_instance",
#    "2022-10-26_22-47-16-248740_instance",
#    "2022-10-26_22-47-16-269873_instance"
#]

#run_ids = [
#    "2022-10-28_19-14-32-769529_instance",
#    "2022-10-28_19-14-32-956637_instance",
#    "2022-10-28_19-14-32-976986_instance",
#    "2022-10-28_19-14-33-075237_instance"
#]

#run_ids = [
#    "2022-10-28_19-41-59-308270_instance",
#    "2022-10-28_19-41-59-318229_instance",
#    "2022-10-28_19-41-59-345927_instance",
#    "2022-10-28_19-41-59-353976_instance"
#]

#run_ids = [
#    "2022-10-28_19-41-59-440573_instance",
#    "2022-10-28_19-41-59-467479_instance",
#    "2022-10-28_19-41-59-566238_instance",
#    "2022-10-28_19-41-59-659763_instance"
#]

#run_ids = [
#    "2022-11-01_14-58-48-404463_instance",
#    "2022-11-01_14-58-48-417278_instance",
#    "2022-11-01_14-58-48-450761_instance"
#]

#run_ids = [
#    "2022-11-02_20-42-43-876346_instance",
#    "2022-11-02_20-42-43-908537_instance",
#    "2022-11-02_20-42-44-186440_instance",
#    "2022-11-02_20-42-44-252045_instance"
#]

#run_ids = [
#    "2022-11-02_20-44-02-520548_instance",
#    "2022-11-02_20-44-02-527643_instance",
#    "2022-11-02_20-44-02-588901_instance",
#    "2022-11-02_20-44-02-592916_instance"
#]

#run_ids = [
#    "2022-11-03_11-40-12-135697_instance",
#    "2022-11-03_11-40-12-143871_instance",
#    "2022-11-03_11-40-12-148828_instance",
#    "2022-11-03_11-40-12-149028_instance"
#]

#run_ids = [
#    "2022-11-03_11-42-58-005356_instance",
#    "2022-11-03_11-42-58-007562_instance",
#    "2022-11-03_11-42-58-017703_instance",
#    "2022-11-03_11-42-58-041547_instance"
#]

#run_ids = [
#    "2022-11-05_11-13-08-503833_instance",
#    "2022-11-05_11-13-08-522633_instance",
#    "2022-11-05_11-13-08-576614_instance",
#    "2022-11-05_11-13-08-581356_instance"
#]

run_ids = [
    "2022-11-05_11-13-10-660672_instance",
    "2022-11-05_11-13-10-753717_instance",
    "2022-11-05_11-13-10-754132_instance",
    "2022-11-05_11-13-10-761661_instance"
]

In [ ]:
with open(f"sbi_data/inference_data/{run_ids[0]}.pkl", "rb") as f:
    instance_params = pickle.load(f)
    simulation_params = instance_params["simulation_params"]

In [ ]:
# Connectivity
if simulation_params["connectivity"] == "Own":
    connectivity = Connectivity()
    connectivity.weights = np.array([[0., 2.], [2., 0.]])
    connectivity.region_labels = np.array(["R1", "R2"])
    connectivity.centres = np.array([[0.1, 0.1, 0.1], [0.2, 0.1, 0.1]])
    connectivity.tract_lengths = np.array([[0., 2.5], [2.5, 0.]])
    connectivity.configure()

# Model
oscillator_model = getattr(tvb.simulator.models, simulation_params["model"])(
    a=np.asarray([simulation_params["a_sim"]]),
    b=np.asarray([simulation_params["b_sim"]]),
    c=np.asarray([simulation_params["c_sim"]]),
    d=np.asarray([simulation_params["d_sim"]]),
    I=np.asarray([simulation_params["I_sim"]]),
)
oscillator_model.configure()

# Integrator
integrator = getattr(tvb.simulator.integrators, simulation_params["integrator"])(dt=simulation_params["dt"])
integrator.noise.nsig = np.array([simulation_params["nsig"]])
integrator.configure()

# Global coupling
coupling = getattr(tvb.simulator.coupling, simulation_params["coupling"])()

# Monitor
monitor = getattr(tvb.simulator.monitors, simulation_params["monitor"])()

In [ ]:
# Simulator
sim = Simulator(
    model=oscillator_model,
    connectivity=connectivity,
    coupling=coupling,
    integrator=integrator,
    monitors=(monitor,),
    simulation_length=simulation_params["simulation_length"]
)

sim.configure();

In [ ]:
X = instance_params["obs"]

In [ ]:
f1 = plt.figure(figsize=(14,8))
plt.plot(X[:, 0, 0, 0], label="Region 1")
plt.plot(X[:, 0, 1, 0], label="Region 2")
plt.ylabel("states", fontsize=16)
plt.xlabel("time (ms)", fontsize=16)
plt.legend(fontsize=16)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.show()

### SNPE inference

In [ ]:
snpe_model = sbiModel(
    method="SNPE",
    obs=X,
    simulator_instance=sim
)

In [ ]:
snpe_model.load(f"{run_ids[0]}.pkl")

In [ ]:
print("Priors\n------")
for iprior in snpe_model.prior.identifier:
    print(f"{iprior[2]}.{iprior[1]}:", "\t", "\t",
          "mean:", np.array(snpe_model.prior.location)[iprior[0]], "\t",
          "std:", np.array(snpe_model.prior.scale)[iprior[0]])

In [ ]:
snpe_model.plot_posterior_samples(
    init_params={"a_model": simulation_params["a_sim"], 
                 "b_model": simulation_params["b_sim"],
                 "c_model": simulation_params["c_sim"],
                 #"d_model": simulation_params["d_sim"],
                 "I_model": simulation_params["I_sim"],
                 "a_coupling": 0.1,
                 "nsig_integrator.noise": 0.003,
                 "noise_global": 0.0}
)

In [ ]:
print("posterior std dev:", snpe_model.posterior_samples.std(dim=0).numpy())
print("prior std dev:", torch.diag(snpe_model.priors.scale_tril).numpy())
print("shrinkages:", snpe_model.posterior_shrinkage().numpy())

In [ ]:
posterior_zscores = snpe_model.posterior_zscore(
    init_params={"a_model": simulation_params["a_sim"],
                 #"b_model": simulation_params["b_sim"],
                 # "c_model": simulation_params["c_sim"],
                 # "d_model": simulation_params["d_sim"],
                 # "I_model": simulation_params["I_sim"],
                 "a_coupling": 0.1,
                 "nsig_integrator.noise": 0.003,
                 "epsilon_global": 0.0}
)
posterior_zscores

In [ ]:
posterior_shrinkages = snpe_model.posterior_shrinkage()
posterior_shrinkages

In [ ]:
f2 = plt.figure(figsize=(12,8))
plt.plot(posterior_shrinkages, posterior_zscores,
         color="blue", linewidth=0, marker="*", markersize=12)
plt.xlabel("posterior shrinkage")
plt.ylabel("posterior zscore")
plt.xlim([-1.6, 1.1])
plt.ylim([0.0, 2])
plt.show();

In [ ]:
posterior_sample = snpe_model.get_sample()[0]
posterior_sample

In [ ]:
# posterior_sample = torch.tensor([2.0, -10.0, 0.0, 0.0, 0.1, 0.003, 0.00001])
# posterior_sample

In [ ]:
posterior_obs = snpe_model.simulation_wrapper(params=posterior_sample)
posterior_obs = posterior_obs.numpy().reshape(X.shape, order="F")

In [ ]:
f3, axes3 = plt.subplots(nrows=2, ncols=1, figsize=(18,15))
axes3[0].plot(snpe_model.inference_data.observed_data.x_obs.values[:, 0, 0, 0], label="R1_observed", color="blue")
axes3[0].plot(posterior_obs[:, 0, 0, 0], label="R1_posterior", color="cyan")
axes3[0].legend(fontsize=16)
axes3[0].set_xlabel("time (ms)", fontsize=16)
axes3[0].tick_params(axis="both", labelsize=16)

axes3[1].plot(snpe_model.inference_data.observed_data.x_obs.values[:, 0, 1, 0], label="R2_observed", color="red")
axes3[1].plot(posterior_obs[:, 0, 1, 0], label="R2_posterior", color="orange")
axes3[1].legend(fontsize=16)
axes3[1].set_xlabel("time (ms)", fontsize=16)
axes3[1].tick_params(axis="both", labelsize=16)

plt.show()

In [ ]:
#snpe_model.save()